In [1]:
import re
import numpy as np
import pandas as pd 
import os
import sys
import itertools
import io

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.utils import *

### Establishes General Path

In [3]:
##local_path = "/your/path/here"
local_path = "/users/thudson/Documents/Dropbox/chicago-police-data/02-column_std"

In [4]:
path = '/Data/chicago-police-data/import'
db = dropbox_handler()
db.list_files(path+'/output')

['incomplete',
 'TRRs',
 'complaints-cpd-2016-dec',
 'complaints-cpd-2016-jun',
 'complaints-cpd-2016-nov',
 'complaints-cpd-2016-oct',
 'complaints-ipra-2016-apr',
 'complaints-merged-2015',
 'shootings-cpd-feb2016',
 'shootings-ipra-may2016']

In [5]:
in_path = path + '/output/complaints-merged-2015/'
out_path = local_path + '/output/complaints-merged-2015/'

In [6]:
try:
    os.mkdir(out_path)
except:
    print("path exists: {}".format(out_path))

path exists: /users/thudson/Documents/Dropbox/chicago-police-data/02-column_std/output/complaints-merged-2015/


In [7]:
##doc_path = '/Data/chicago-police-data/02-column_std'
##column_dict = db.download_file(doc_path+'/doc/','Column_Dictionary.csv')

doc_path = local_path
column_dict = pd.read_csv(doc_path+'/doc/'+'Column_Dictionary.csv').iloc[:,0:3]
column_dict.head()

,Orig Column Name,New Name,Alternative Name
0,Victim_Age,Civilian.Victim.Age,NaN
1,Victim_Gender,Civilian.Victim.Gender,NaN
2,Victim_Race,Civilian.Victim.Race_Code,NaN
3,Witness_Age,Civilian.Witness.Age,NaN
4,Witness_Gender,Civilian.Witness.Gender,NaN


In [8]:
## SOMETHING TO PUT IN UTILS? HOW WOULD THE SPECIFIED COLUMN_DICT WORK?

def column_mapper(df,
                  column_dict = column_dict):
    current_columns = pd.DataFrame(df.columns.tolist(),columns=['Orig Column Name'])
    new_columns = current_columns.merge(column_dict,how='left',on='Orig Column Name')['New Name'].tolist() 
    if current_columns.shape[0] == len(new_columns):
        return new_columns
    else:
        print("Join failed, more columns returned than expected")

## March 2015 Data

In [9]:
in_path_mar_2015_report = in_path+'march_2015'
db.list_files(in_path_mar_2015_report)


['Report_1_-_All_Complaints_in_Time_Frame',
 'Report_2_-_Identified_Accused',
 'Report_3_-_Police_Witnesses',
 'Report_4_-_Complaining_Witnesses']

### Report 1

In [10]:
in_path_mar_2015_report = in_path+'march_2015/Report_1_-_All_Complaints_in_Time_Frame'
out_path_mar_2015_report = out_path

files = db.list_files(in_path_mar_2015_report)
files

['foia_14-5509_-_report_1.csv',
 'foia_14-5509_-_report_1.xlsx',
 'foia_14-5509_-_report_1_metadata.csv']

In [11]:
file = files[0]
df = db.download_file(in_path_mar_2015_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-merged-2015/march_2015/Report_1_-_All_Complaints_in_Time_Frame/foia_14-5509_-_report_1.csv


['CRID',
 'Beat',
 'Location_Code',
 'Location_Value',
 'Address_of_Incident',
 'City',
 'State',
 'Zip',
 'Incident_Date',
 'Complaint_Date',
 'Closed_Date',
 'Investigator',
 'FOIA_Request_Number']

In [12]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Incident.Location.Beat',
 'Incident.Location.Location_Code',
 'Incident.Location.Location_Value',
 'Incident.Location.Address',
 'Incident.Location.City',
 'Incident.Location.State',
 'Incident.Location.Zip',
 'Incident.Time.Date',
 'Complaint.Date',
 'Complaint.Current.Closed_date',
 'Investigator.Full_Name',
 'FOIA_Request_Number']

In [13]:
df.to_csv(out_path_mar_2015_report+'mar_2015'+file,index=False)

### Report 2

In [14]:
in_path_mar_2015_report = in_path+'march_2015/Report_2_-_Identified_Accused'
out_path_mar_2015_report = out_path

files = db.list_files(in_path_mar_2015_report)
files

['foia_14-5509_-_report_2.csv',
 'foia_14-5509_-_report_2.xlsx',
 'foia_14-5509_-_report_2_metadata.csv']

In [15]:
file = files[0]
df = db.download_file(in_path_mar_2015_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-merged-2015/march_2015/Report_2_-_Identified_Accused/foia_14-5509_-_report_2.csv


['CRID',
 'Accused',
 'Accused_Gender',
 'Accused_Race_Code',
 'Date_of_Appt',
 'Current_Unit',
 'Current_Rank',
 'Star',
 'Complaint_Category',
 'Orig_Finding',
 'Orig_Recommended_Discipline',
 'Final_Finding',
 'Final_Recommended_Discipline',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [16]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Officer.Accused.Full_Name',
 'Officer.Accused.Gender',
 'Officer.Accused.Race_Code',
 'Officer.Accused.Appointed_Date',
 'Officer.Accused.Unit',
 'Officer.Accused.Rank',
 'Officer.Accused.Star',
 'Complaint.Current.Category',
 'Complaint.Initial.Finding',
 'Complaint.Initial.Recommended_Discipline',
 'Complaint.Final.Finding',
 'Complaint.Final.Recommended_Discipline',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [17]:
df.to_csv(out_path_mar_2015_report+'mar_2015'+file,index=False)

### Report 3

In [18]:
in_path_mar_2015_report = in_path+'march_2015/Report_3_-_Police_Witnesses'
out_path_mar_2015_report = out_path

files = db.list_files(in_path_mar_2015_report)
files

['foia_14-5509_-_report_3.csv',
 'foia_14-5509_-_report_3.xlsx',
 'foia_14-5509_-_report_3_metadata.csv']

In [19]:
file = files[0]
df = db.download_file(in_path_mar_2015_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-merged-2015/march_2015/Report_3_-_Police_Witnesses/foia_14-5509_-_report_3.csv


['CRID',
 'Officer_Witness',
 'Officer_Witness_Gender',
 'Officer_Witness_Race',
 'Officer_Witness_Star',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [20]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Officer.Witness.Full_Name',
 'Officer.Witness.Gender',
 'Officer.Witness.Race_Code',
 'Officer.Witness.Star',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [21]:
df.to_csv(out_path_mar_2015_report+'mar_2015'+file,index=False)

### Report 4

In [22]:
in_path_mar_2015_report = in_path+'march_2015/Report_4_-_Complaining_Witnesses'
out_path_mar_2015_report = out_path

files = db.list_files(in_path_mar_2015_report)
files

['foia_14-5509_-_report_4.csv',
 'foia_14-5509_-_report_4.xlsx',
 'foia_14-5509_-_report_4_metadata.csv']

In [23]:
file = files[0]
df = db.download_file(in_path_mar_2015_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-merged-2015/march_2015/Report_4_-_Complaining_Witnesses/foia_14-5509_-_report_4.csv


['CRID',
 'Witness_Gender',
 'Witness_Race',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [24]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Civilian.Witness.Gender',
 'Civilian.Witness.Race_Code',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [25]:
df.to_csv(out_path_mar_2015_report+'mar_2015'+file,index=False)